In [9]:
import torch
import matplotlib.pyplot as plt

import utils.datasets as datasets
import utils.sde_lib as sdes
from utils.models import MLP
from utils.misc import dotdict 

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = 1000
# covs = torch.tensor([[[3., -1.],[-1., 2.]], [[3., 1.8],[1.8, 2.]], [[3., -1.],[-1., 2.]]], device=device)
# means = torch.tensor([[0,3.], [10,5.], [-6.,0]],device=device)
# weights = torch.ones(means.shape[0],device=device)/means.shape[0]
# gmm = GMM(weights,means,covs)
dataset = datasets.get_dataset(dotdict({'dataset' : 'spiral'}))
data = dataset.sample(5).to(device=device)

sde = sdes.LinearSchrodingerBridge(2,device)
model = MLP(2,False).to(device=device)

eps = sde.delta
times = (torch.rand((data.shape[0]),device=data.device) * (1-eps) + eps) * sde.T()
shaped_t = times.reshape(-1,1,1,1) if len(data.shape) > 2 else times.reshape(-1,1)
mean, L, invL = sde.marginal_prob(data,shaped_t)
noise = torch.randn_like(mean,device=data.device)
perturbed_data = mean + torch.bmm(L, noise.unsqueeze(-1)).squeeze(-1)
flatten_error = ((torch.bmm(invL.mT, noise.unsqueeze(-1)).squeeze(-1) + model(perturbed_data,times))**2).view(data.shape[0],-1)


std = (1-torch.exp(-sde.beta_int(shaped_t)))**.5

# print(times)
# print((std).cpu().numpy())
# print(L[:,0,0].cpu().numpy())

# print(L)
# print(invL.mT)

# print(perturbed_data)
# print(torch.exp(-sde.beta_int(shaped_t)/2) * data + noise * std)

print(torch.bmm(invL.mT, noise.unsqueeze(-1)).squeeze(-1))
print(noise / std)

# plt.scatter(data[:,0], data[:,1])
# plt.show()



tensor([[ -0.8638,   0.3381],
        [-17.8266,  -9.2867],
        [ -0.7868,  -0.1076],
        [  2.1691,  -1.3685],
        [  0.0546,  -2.2348]], device='cuda:0')
tensor([[ -0.8641,   0.3383],
        [-17.8268,  -9.2868],
        [ -0.7870,  -0.1076],
        [  2.1697,  -1.3688],
        [  0.0546,  -2.2354]], device='cuda:0')
